ft-make data work


input

<user>what is the capital of France?</user>
<assistant>Paris</assistant>
<user>what about Spain?</user>
<assistant>Madid</assistant>
<user>Germany?</user>

target output
berlin


Input Alice has 3 apples and buys 2 more. How many now?

Target Output

<think> Start with 3. Buys 2 ⇒ 3+2=5. </think>

<answer>5</answer>


Input 
Carly has 8 apples and buys 2 more, but then sells 5 to the local baker. How many now?

Model Output 
8+2-5=5 
Answer: 5

Grader 
Correct: +1 
Shows work: +1 
Total reward (score): +2

Input
Greet politely

Model Output
Hi there! How are you?

Grader
High score on politeness: +1
High score on enthusiasm: +1
High score on engagement: +1
Total reward (score): +3


In [ ]:
dataset = load_dataset('openai','main')
train = dataset['train']
test = dataset['test']

In [ ]:
input_ids = tokenizer(text)

output_text = tokenizer.decode(output_text)

In [ ]:
#from probabilities to next token id

model.generate(**ids) #greedy selection

model.generate(**ids,do_sample=true,temperature) #sample
# temperature = 0 时 就相当于greedy selection

#beam search

model.generate(**ids , num_beams=2)


In [ ]:
AutoTokenizer.from_pretrained(module_name)(prompt_batch , padding=True)['input_ids']


tensor([[0,  0,  2640,   3073,    418,    3221,  21142],
        [0,  0,  3073,    418,   3221,    21142,    30],
        [0,  0,     0,      0,   2640,    3073,    418]])

In [ ]:
#加掩码算损失

import torch
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss(ignore_index = -100) #遇到数字-100 直接跳过

logits = torch.randn(1,6,1000) # batch_size seq_len vocab_size

labels = torch.tensor([[-100, -100, -100,  -100,  45,  982]])

loss = loss_fn(logits.view(-1,1000),  labels.view(-1))
#.view()  重塑张量形状 
# view(-1请pytorch自行计算剩下的维度合并起来时多少,词表大小)处理输出logits
#view(-1 把所有元素拉成一排)处理标签
loss.backward()


In [ ]:
training_args = SFTConfig(completion_only_loss = True , ...)

trainer = SFTTrainer(
    model = model_name,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
)

trainer.train()

In [ ]:
training_args = TrainingArguments(
    learning_rate = 2e-6,
    weight_decay = 0.01,
    per_device_train_batch_size = 20,
    per_device_eval_batch_size = 20,
    num_train_epochs = 3,
)

In [ ]:
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)           # 为当前 GPU 设置种子
    torch.cuda.manual_seed_all(seed)       # 为所有 GPU 设置种子（多卡训练）
    
    # 强制 CuDNN 使用确定性算法（会让训练变慢，但保证结果完全一致）
    torch.backends.cudnn.deterministic = True 
    torch.backends.cudnn.benchmark = False
    

固定随机种子

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import LoraModel , LoraConfig

config = LoraConfig(
    task_type = "SEQ_2_SEQ_LM",
    r = 8,
    lora_alpha = 32,
    target_modules = ["q" , "v"]
)

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
lora_model = LoraModel(model,config,"default")


In [ ]:
#verifiers in GRPO and DeepSeek models

def math_reward(response, correct_answer):
    if (extract_final_answer(response) == correct_answer):
        return 1.0
    else:
        return 0.0


def format_reward(response):
    if ("<think>" in response and "</think>" in response):
        return 1.0
    else:
        return 0.0
    
def language_consistency_reward(response):
    if consistent_language(response):
        return 1.0
    else:
        return -0.5
    

def combined_reward(response, correct_answer):
    accuracy = math_reward(response, correct_answer)
    format_check = format_reward(response)
    language_check = language_consistency_reward(response)

    return accuracy + format_check + language_check


In [ ]:
if predicted == correct_answer :
    reward = 1.0
else:
    relative_error = abs(predicted - correct_answer)/correct_answer

    if relative_error < 0.01:
        reward = 1.0
    elif relative_error < 0.1:
        reward = 0.8
    elif relative_error < 0.3:
        reward = 0.5

In [ ]:
GRPO_config = GRPOConfig(
    num_generations = 12
    temperature = 0.7
)

temperature helps you sample different outputs,per input

In [ ]:
mean_reward = sum(group_rewards) / len(group_rewards)
normalized_rewards = [r - mean_reward for r in group_rewards]


In [ ]:
import math

def compute_rewards(prompts, completions):
    #同一个问题prompt 对应 不同回答completions

    unique_prompts = list(set(prompts))

    for i, prompt in enumerate(unique_prompts): #unique_prompts是prompts进行了去重操作
        prompt_indices = [i for i,p in enumerate(prompts) if p == prompt]
        group_outputs = [completions[i] for i in prompt_indices] 
    for j,out in enumerate(group_outputs):
        reward = reward_model.compute_reward(out,correct_out,prompt)
        group_rewards.append(reward)

    mean_reward = sum(group_rewards) / len(group_rewards)
    variance = sum((r - mean_reward) ** 2 for r in group_rewards)/len(group_rewards)
    std_reward = math.sqrt(variance)
    normalized_rewards = [(r - mean_reward)/(std_reward + 1e-8) for r in group_rewards]


In [ ]:
trainer = GRPOTrainer(
    model = model ,
    reward_funcs = compute_rewards, #奖励函数
    args = GRPO_config,
    train_dataset = eval_dataset,
    processing_class = tokenizer,
)

trainer.train()

In [ ]:
suite: "debug_search_api_v1"    #held out RL test env
version: "1.0.0"        #env版本
global_seed: 424242         #fixed RNG for reproducibility
time_fixed_utc:"2026-01-21T12:00:00Z"       #freeze time inside the env

llm:
    model:"policy-llm@sha256:POLICY_SHA"  #指定模型 使用哈希 比特级一致
    prompt_template_sha:"SHA_PROMPT_V1"  #指定prompt模板
    tokenizer_sha:"SHA_TOKENIZER_V1"    #指定tokenizer
    decoding:       #deterministic decoding
        temperature: 0.0    #greedy search
        top_p: 0.0      #greedy search 配合temperature使用
        top_k: 0        #greedy search 配合temperature使用
        max_new_tokens: 512 #生成长度限制 控制显存和时间 防止进入死循环
    determinism:
        cuda_deterministic: True  #确保cuda操作确定性
        cuda_benchmark: False


tools:
    - name: "search_api"
      interface: "openapi" #工具接口标准
      mode: "replay"       #工具使用模式 replay回放
      fixture: "fixtures/search_api_v3.json"    #指定“剧本”文件
      seed: 1337
    - name: "my_codebase"
      interface: "filesystem" #工具接口标准
      mode: "snapshot"
      snapshot_sha: "GIT_SHA_CODEBASE_V3"
      mount_path: "/mnt/code"

verifiers:      # programmatic graders
    unit_tests_v2:
        type: "code_tests"
        config: "verifiers/unit_tests_v2.yaml"  # which tests to run/expect
    api_version_checker_v1:
        type: "string_match"
        expected_version: "3.2.1"

RL env

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans 

embedding_model = SetenceTransformer('all-MiniLM-L6-v2')

embeddings = embedding_model.encode(texts)

kmeans = KMeans(n_clusters=3 , random_state = 42)
cluster_labels = kmeans.fit_predict(embeddings)

聚类进行error analysis

In [ ]:
EEROR_CATEGORIES = {
    "calculation_error",
    "reasoning_error",
    "incomplete_solution",
    "format_error",
    "other"
}

prompt = f"""Analyze this math error and categorize it:
PROBLEM: {question}...
CORRECT: {correct_answer}
PREDICTED: {predicted_answer}
Choose ONE category: {', '.join(ERROR_CATEGORIES)}
Respond with just the category name."""


SyntaxError: incomplete input (2183637626.py, line 9)

利用python自动填表，生成给LLM的prompt